In [123]:
# https://www.kaggle.com/code/rishavdash/hand-number-classification-mediapipe-tutorial   => mediapipe

# import libraries
import cv2
import numpy as np
import mediapipe as mp
import os


In [124]:
# holistic model
mp_holistic = mp.solutions.holistic

# Initialize mediapipe drawing utilities
mp_drawing = mp.solutions.drawing_utils

In [125]:
def detection (img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert the input into RGB format to pass to mediapipe
    img.flags.writeable = False # To improve performance, mark the image as not writeable
    results = model.process(img) # detection
    img.flags.writeable = True
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # convert the input into BGR format to pass to opencv imshow
    return img, results

In [131]:
def draw_landmarks (img, results):
    mp_drawing.draw_landmarks(img, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,128,0), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(img, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,51,153), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(img, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,51,153), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))

In [127]:
# get webcam
cap = cv2.VideoCapture('./videoplayback.mp4')
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        ret, frame = cap.read()

        img, results = detection(frame, holistic) # make detection
        draw_landmarks(img,results) # draw landmarks

        cv2.imshow('Human Pose Estimation', img)
        if cv2.waitKey(10) & 0xFF == ord('e'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [128]:
if results.pose_landmarks.landmark:
    for i in range(0,32):
        print(f'{mp_holistic.PoseLandmark(i).name}:\n{results.pose_landmarks.landmark[mp_holistic.PoseLandmark(i).value]}') 

NOSE:
x: 0.4179377
y: 0.34437203
z: -0.8218116
visibility: 0.9999971

LEFT_EYE_INNER:
x: 0.43264803
y: 0.33133027
z: -0.80030036
visibility: 0.99999297

LEFT_EYE:
x: 0.44079313
y: 0.33097
z: -0.8001473
visibility: 0.9999918

LEFT_EYE_OUTER:
x: 0.45035756
y: 0.33085996
z: -0.8002552
visibility: 0.999993

RIGHT_EYE_INNER:
x: 0.40337434
y: 0.332124
z: -0.8007532
visibility: 0.9999963

RIGHT_EYE:
x: 0.39407483
y: 0.33218127
z: -0.8009933
visibility: 0.9999957

RIGHT_EYE_OUTER:
x: 0.38585892
y: 0.33228168
z: -0.800988
visibility: 0.99999595

LEFT_EAR:
x: 0.46456224
y: 0.3344797
z: -0.61842424
visibility: 0.9999891

RIGHT_EAR:
x: 0.37539873
y: 0.33643636
z: -0.6240335
visibility: 0.9999896

MOUTH_LEFT:
x: 0.43663967
y: 0.35779414
z: -0.7472945
visibility: 0.9999947

MOUTH_RIGHT:
x: 0.4031764
y: 0.35839838
z: -0.74908155
visibility: 0.999997

LEFT_SHOULDER:
x: 0.53037447
y: 0.41147885
z: -0.4687145
visibility: 0.9998173

RIGHT_SHOULDER:
x: 0.3079478
y: 0.41440275
z: -0.46929115
visibility: 0.

In [118]:
# extracting body keypoints
def keypoint_extraction(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hand = np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*4)
    right_hand = np.array([[res.x, res.y, res.z, res.visibility] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*4)
    return np.concatenate([pose, left_hand, right_hand])

In [122]:
# declare actions
actions = ['like', 'dislike', 'heart', 'ok', 'thanks']
vids_num = 50
vid_length = 30 #frames

# create data directory
for action in actions:
    for vid_num in range(vids_num):
        os.makedirs(os.path.join('data', action, str(vid_num)))